In [1]:
import sys
from pprint import pprint, pformat  # NOQA
from optparse import OptionParser
from hydra.kafkatest.runtest import RunTestKAFKA

In [2]:
class Options():
    test_duration = 10
    msg_batch = 100        
    msg_rate= 2000
    total_sub_apps = 1
    config_file = 'hydra.ini'
    keep_running = False

In [3]:
global_results = dict()
test_instance = None
options = Options()
runner = RunTestKAFKA(options, False)      #Initialize ALL variables (from CLI options and hydra.ini config file)
first_test = False

ERROR 2016-05-31 18:07:07,611 runtestbase.py:__init__:52 Unable to open config file hydra.ini


Exception: Unable to open config file hydra.ini

In [4]:
def RunTest(test_name, first_test):
    if not first_test:
        runner.start_appserver()
        first_test = True
    else:
        runner.set_options(options)
        runner.scale_sub_app()
    res = runner.run_test()
    print("RES = " + pformat(res))
    if not str(options.total_sub_apps) in global_results:
         global_results[str(options.total_sub_apps)] = {}
    global_results[str(options.total_sub_apps)][test_name] = res


In [ ]:
#client_set = [1, 5, 10, 20, 25, 30]
#msg_rate_set = [500, 1000, 2000, 3000, 4000, 5000, 8000]
client_set = [10, 20]
msg_rate_set = [1000, 1500, 2000]
first_test = False

for client_num in client_set:
    options.total_sub_apps = int(client_num / 10)
    for msg_rate in msg_rate_set:
        options.msg_rate = msg_rate
        RunTest('run_' + str(msg_rate), first_test)
        first_test = True
        #print("Run_%s FINISHED" %str(msg_rate))
        #wait = raw_input("PRESS ENTER TO CONTINUE.")
        #print("Next Run to be executed...")

In [ ]:
client_set = [20, 50]
msg_rate_set = [1000, 1500]
first_test = False

for client_num in client_set:
    options.total_sub_apps = int(client_num / 10)
    for msg_rate in msg_rate_set:
        options.msg_rate = msg_rate
        RunTestBulk('run_' + str(msg_rate), first_test)
        first_test = True
        print("Run_%s FINISHED" %str(msg_rate))
        wait = raw_input("PRESS ENTER TO CONTINUE.")
        print("Next Run to be executed...")

In [8]:
import plotly.plotly as py
from plotly.graph_objs import *

traces_plot1 = []
traces_plot2 = []
traces_plot3 = []

for trace, tests_per_trace in global_results.iteritems():
    average_packet_loss = []
    average_packets = []
    average_rate = []
    average_tx_rate = []
    client_count = []
    failing_clients = []
    failing_clients_rate = []
    packet_tx = []
    pub_net_rxrate = []
    pub_net_txrate = []
    for test, stats in tests_per_trace.iteritems():
        average_packet_loss.append(stats['average_packet_loss'])
        average_packets.append(stats['average_packets'])
        average_rate.append(stats['average_rate'])
        average_tx_rate.append(stats['average_tx_rate'])
        client_count.append(stats['client_count'])
        failing_clients.append(stats['failing_clients'])
        if stats['failing_clients'] != 0:
            failing_clients_rate.append(stats['failing_clients_rate'])
        packet_tx.append(stats['packet_tx'])
        pub_net_rxrate.append(stats['pub_net_rxrate'])
        pub_net_txrate.append(stats['pub_net_txrate'])
        
    trace_plot1 = Scatter(
      x=average_tx_rate,
      y=average_rate
    )
    trace_plot2 = Scatter(
      x=average_tx_rate,
      y=average_packet_loss
    )
    trace_plot3 = Scatter(
      x=average_tx_rate,
      y=failing_clients_rate
    )
    
    traces_plot1.append(trace_plot1)
    traces_plot2.append(trace_plot2)
    traces_plot3.append(trace_plot3)
    


In [9]:
data = Data(traces_plot1)
py.iplot(data, filename = 'kafka-rx-tx')

/home/annyz/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



In [10]:
data = Data(traces_plot2)
py.iplot(data, filename = 'kafka-loss-tx')

/home/annyz/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



In [12]:
data = Data(traces_plot3)
py.iplot(data, filename = 'kafka-fail-tx')

/home/annyz/venv/local/lib/python2.7/site-packages/requests/packages/urllib3/util/ssl_.py:122: InsecurePlatformWarning:

A true SSLContext object is not available. This prevents urllib3 from configuring SSL appropriately and may cause certain SSL connections to fail. You can upgrade to a newer version of Python to solve this. For more information, see https://urllib3.readthedocs.org/en/latest/security.html#insecureplatformwarning.



In [ ]:
runner.delete_all_launched_apps()
runner.stop_appserver()